In [10]:
import h5py
import numpy as np
from numpy.random import random

##### Filesize experiments

In [11]:
dset1 = random( size=(1000,773) )
dset2 = np.round( random( size=(1000,773) ) ).astype(bool)

In [16]:
with h5py.File("data/float.hdf5") as f:
    f.create_dataset("float", data=dset1, maxshape=(None, 773))

with h5py.File("data/bool.hdf5") as f:
    f.create_dataset("bool", data=dset2, dtype=bool, maxshape=(None, 773), compression="gzip", compression_opts=9)
